# Initial processing of Understanding Society data

Starting off with the full Understanding Society dataset (Ind+hh_with_xwave_all_waves.dta), I made the following changes in Stata:

1. Start with 505,477 rows and 8,488 columns
2. Drop all irrelevant waves (294,270 observations deleted)
3. Dropped all vars with >10% missing (7,603 columns dropped)
4. Dropped all rows where nbrsnci_dv < 0 or scghq1_dv < 0 (35,872 rows dropped) (17%)
5. Dropped all informal institutions and SW variables except nbrsnci_dv and scghq1_dv (21 columns dropped)
6. Dropped vars with >20% negative variables (521 columns dropped) (343 vars remain)

This notebook will have a look at the large processed dataset (343 vars, 175,335 obs), and run a lasso regression to see which variables are most predictive of subjective wellbeing (measured on the Likert scale).

Before starting, these are the 15 variables I hand selected based on intuition and prior research to include in my refined variable list:

- gor_dv : Government Office Region
- urban_dv : Urban or rural area, derived
- nbrsnci_dv : Buckner's Neighbourhood Cohesion Instrument, short (α= .88)
- scghq1_dv : Subjective wellbeing (GHQ): Likert
- sex_dv : Sex, derived
- age_dv : Age, derived from dob_dv and intdat_dv
- ethn_dv : Ethnic group (derived from multiple sources)
- marstat_dv : Harmonised de facto marital status
- jbstat : Current economic activity
- qfhigh_dv : Highest educational qualification ever reported
- jbiindb_dv : Current job: Industrial classification (CNEF), two digits
- fimnnet_dv : total net personal income - no deductions
- fihhmnnet1_dv : total household net income - no deductions
- houscost1_dv : monthly housing cost including mortgage principal payments
- health : Long-standing illness or disability

In [29]:
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
%matplotlib inline
import pyreadstat

In [30]:
# Import file

path = "/Users/arikatz/VSCode Projects/ukhls-informal-institutions-project/data/droppedvaraibleswithlotsofnegatives.dta"
df_full, meta = pyreadstat.read_dta(path)

print("Shape:", df_full.shape)
df_full.head()

Shape: (175335, 343)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,xpmove,jbstat,racel_dv,health,aidhh,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,fiyrdia,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,hgpart,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,ienddathh,ienddatmm,ienddatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,ndepchl_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhigh_dv,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,b_hidp,b_pno,b_ivfio,b_ivfho,b_month,c_hidp,c_pno,c_ivfio,c_ivfho,c_month,d_hidp,d_pno,d_ivfio,d_ivfho,d_month,e_hidp,e_pno,e_ivfio,e_ivfho,e_month,f_hidp,f_pno,f_ivfio,f_ivfho,f_month,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,h_hidp,h_pno,h_ivfio,h_ivfho,h_month,i_hidp,i_pno,i_ivfio,i_ivfho,i_month,genetics,epigenetics,xwdat_dv,scend_dv,school_dv,bornuk_dv,generation,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
0,22445,f,6,3.4,25,278664010,3,3,3,4,2,1,6,1,11,2,29,1984,26,6,2014,2,1,2,1,2,2,2,1,0,0,0,0,0,0,1,0,2,2,1,2,1,0,0,0,0,0,0,0,0,2,5,2,3,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,2,1,18,16,57,19,7,12,0,0,2572.590088,0,2572.590088,0,2012.00,0,0.0,0,0.000000,90,72.0,2,29,26,6,2014,1984,2,1,6,1,1,0,0,0,1,0.0,0.0,0.0,0.0,0.000000,2012.000000,1,7,1,1,0,6,3,5,0,0,1,1,17,3,2,0,0,2,2,2,272012925,1,0,0,0,0,1,1,0,0,-8,0,3,31,62.12,32.59,1,0,0,0,0,0.00,0.0,0.00,0.0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,14,4,14,4,-9,-9,-9,-9

In [31]:
df = df_full

## Dealing with missing and negative values

Great! Now we need to deal with missing and negative values. 

First, let's see which columns have the most missing and negative values.

Then, we can drop those columns.

Finally, we'll drop any remaining rows with missing or negative values.

In [32]:
# Calculate % missing and % negative for each column
missing_pct = df.isnull().mean() 
negative_pct = (df.select_dtypes(include=['number']) < 0).mean() * 100

# Combine into a summary DataFrame and sort by the sum of % missing and % negative (descending)
summary = pd.DataFrame({
    '% Missing': missing_pct,
    '% Negative': negative_pct
}).fillna(0)

summary['% Total'] = summary['% Missing'] + summary['% Negative']
summary = summary.sort_values(by='% Total', ascending=False)

summary.head(50)

,% Missing,% Negative,% Total
i_pno,0.0,19.098868,19.098868
i_month,0.0,18.786894,18.786894
i_ivfho,0.0,17.531582,17.531582
i_ivfio,0.0,17.531582,17.531582
i_hidp,0.0,17.531582,17.531582
h_pno,0.0,16.072661,16.072661
h_month,0.0,15.732170,15.732170
aidhh,0.0,15.215445,15.215445
ndepchl_dv,0.0,15.139020,15.139020
h_hidp,0.0,14.320586,14.320586


In [33]:
# Delete the vars with total % ineligible >5%
cols_to_drop = summary[summary['% Total'] > 5].index
cols_to_drop = summary.head(25).index
df = df.drop(columns=cols_to_drop)

print("Shape after dropping vars with total % ineligible >5%:", df.shape)
df.head()

Shape after dropping vars with total % ineligible >5%: (175335, 318)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,xpmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,b_hidp,b_pno,b_ivfio,b_ivfho,b_month,c_hidp,c_pno,c_ivfio,c_ivfho,c_month,d_hidp,d_pno,d_ivfio,d_ivfho,d_month,e_hidp,e_ivfio,e_ivfho,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,genetics,epigenetics,xwdat_dv,scend_dv,school_dv,bornuk_dv,generation,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
0,22445,f,6,3.4,25,278664010,3,3,3,4,2,1,6,1,11,2,29,1984,26,6,2014,2,1,2,1,2,2,1,0,0,0,0,0,0,1,2,2,1,2,1,0,0,0,0,0,0,0,0,2,5,2,3,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,2,1,18,16,57,0,0,2572.590088,0,2572.590088,0,2012.00,0,0.0,0,0.000000,90,72.0,2,29,26,6,2014,1984,2,1,6,1,1,0,0,0,1,0.0,0.0,0.0,0.0,0.000000,2012.000000,1,7,1,1,0,6,3,5,0,0,1,1,17,3,2,0,2,2,2,272012925,1,0,0,0,0,1,1,0,0,0,3,31,62.12,32.59,1,0,0,0,0,0.00,0.0,0.00,0.0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,14,4,14,4,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9,280942006,5,1,10,4,279255608,1,10,278447092,1,1,10,6,0,0,3,-8,3,1,5,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999978,0.999948,0.999920,0.999890,0.999854,0.999813,0.999772,0.999738,0.999689,

In [34]:
# Drop rows with any missing values
df = df.dropna()  # Remove rows with any NaN values

print("Shape after dropping rows with missing values:", df.shape)

df.head()

Shape after dropping rows with missing values: (173487, 318)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,xpmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,b_hidp,b_pno,b_ivfio,b_ivfho,b_month,c_hidp,c_pno,c_ivfio,c_ivfho,c_month,d_hidp,d_pno,d_ivfio,d_ivfho,d_month,e_hidp,e_ivfio,e_ivfho,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,genetics,epigenetics,xwdat_dv,scend_dv,school_dv,bornuk_dv,generation,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
1,22445,i,9,3.3,11,277344816,1,3,3,4,2,1,6,1,11,2,33,1984,23,10,2017,2,2,2,1,2,2,2,0,0,0,0,0,0,1,2,3,1,1,1,0,0,0,0,0,0,0,0,4,4,4,5,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,18,50,38,0,0,2423.030029,0,2333.330078,0,1200.000000,0,0.0,0,89.699997,0,0.0,2,33,23,10,2017,1984,2,0,1,0,0,0,0,0,1,0.0,0.0,0.0,0.0,89.699997,1289.699951,1,7,1,1,0,7,4,6,0,0,1,2,3,1,2,0,2,2,2,22445,1,0,0,0,0,0,0,0,0,0,3,31,57.20,46.08,1,0,0,0,0,0.00,0.0,0.00,0.0,10,23,10,2017,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,2,1,1,0,0,0,1,1,2,1,1,1,1,1,1,1,1,1,1,0,1,300,150,30,1,0,0,2423.030029,0,2333.330078,0,1,1289.699951,1200.000000,0.0,0.0,0.0,0.0,89.699997,1300.0,736.869995,2423.030029,4,0,0,0,0,0,0,0,0,1,0,1,0,1,1.0,2,0,0.0000,1,10.00,110,9,0,2,14,4,14,4,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9,280942006,5,1,10,4,279255608,1,10,278447092,1,1,10,6,0,0,3,-8,3,1,5,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999978,0.999948,0.999920,0.999890,0.999854,0.999813,0.999772,0.999738,0.999689,0.999649,0.999609,0.999566,0.999452,0.999389,0.999288,0.999219,0.9

In [35]:
# Drop rows with any negative values
numeric_cols = df.select_dtypes(include='number').columns
df = df[(df[numeric_cols] >= 0).all(axis=1)]
print("Shape after dropping rows with negative values:", df.shape)
df.head()

Shape after dropping rows with negative values: (106771, 318)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,xpmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,b_hidp,b_pno,b_ivfio,b_ivfho,b_month,c_hidp,c_pno,c_ivfio,c_ivfho,c_month,d_hidp,d_pno,d_ivfio,d_ivfho,d_month,e_hidp,e_ivfio,e_ivfho,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,genetics,epigenetics,xwdat_dv,scend_dv,school_dv,bornuk_dv,generation,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
8,280165,i,9,3.3,7,754793216,1,3,3,67,15,1,12,1,11,2,38,1979,31,1,2018,2,2,2,1,1,2,2,0,0,0,0,0,0,1,2,1,2,3,1,1,1,1,1,0,1,1,1,5,5,3,6,2,1,2,0,0,1,0,0,0,0,0,2,0,2,0,4,1,19,17,37,0,0,3089.699951,0,3000.0,0,2228.000000,0,0.000000,0,89.699997,0,0.0,2,38,31,1,2018,1979,2,1,1,1,1,0,0,0,1,0.0,0.0,0.00,0.0,89.699997,2317.699951,1,8,2,1,0,8,4,7,1,0,0,2,20,1,2,1,1,2,1,783876922,1,4,4,0,0,2,2,0,0,1,4,16,55.57,52.43,1,0,0,0,0,0.0,0.0,0.0,0.0,10,31,1,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2,2,1,1,0,1,0,1,1,2,1,1,1,1,1,1,1,1,1,1,0,1,500,100,0,3,0,1,10162.950195,0,8228.730469,0,1,8086.029785,6151.810059,0.0,0.0,9.560000,1184.959961,739.700012,1536.0,1536.000000,10162.950195,5,1,0,1,0,0,1,0,1,2,1,2,1,3,2.3,2,0,0.6242,1,6.00,110,7,0,2,14,2,14,2,783876802,2,1,10,11,759532804,2,2,11,11,758492406,2,2,11,11,756833208,1,10,755847212,1,1,10,12,0,0,3,15,1,1,6,1,1,1,1,0.000342,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.999982,0.999959,0.999937,0.999912,0.999884,0.999851,0.999818,0.999791,0.999752,0.999720,0.999688,0.99965

## Separating outcome variable and dropping identifiers

Great! Now we need to take out scghq1_dv (the outcome variable) and drop identifiers (pidp wave wave_num hidp)

In [41]:
# Separate outcome (scghq1_dv) and predictors (X)

Y = df['scghq1_dv'].astype(float)  # Ensure Y is a numeric pandas Series
X = df.drop(columns=['scghq1_dv'])

# Drop identifiers (pidp, wave, wave_num, hidp, pno, hhorig, memorig, psu, strata, sampst, month, ivfio, ioutcome, birthy, hrpid)
identifiers = ['pidp', 'wave', 'wave_num', 'hidp', 'pno', 'hhorig', 'memorig', 'psu', 'strata', 'sampst', 'month', 'ivfio', 'ioutcome', 'birthy', 'hrpid']
X = X.drop(columns=identifiers)

## Defining categorical and continuous variables

In [37]:
# Strict rule: discrete only if dtype is object/category or integer dtype, and num_unique <= MAX_UNIQUE_DISCRETE
# Anything with more than MAX_UNIQUE_DISCRETE unique values is automatically continuous.

MAX_UNIQUE_DISCRETE = 30

label_map = dict(zip(meta.column_names, meta.column_labels)) if hasattr(meta, 'column_names') else {}
rows = []

for col in X.columns:
    dtype = X[col].dtype
    col_vals = X[col].dropna()
    num_unique = int(col_vals.nunique())

    # Automatic continuous if too many uniques
    if num_unique > MAX_UNIQUE_DISCRETE:
        will_discrete = False
        reason = f'{num_unique} unique > {MAX_UNIQUE_DISCRETE} -> continuous'
    else:
        is_string = dtype == 'object' or str(dtype).startswith('category')
        is_integer = pd.api.types.is_integer_dtype(X[col].dtype)

        will_discrete = bool(is_string or is_integer)
        if is_string:
            reason = 'string/category dtype -> discrete'
        elif is_integer:
            reason = 'integer dtype -> discrete'
        else:
            reason = 'float/numeric dtype -> continuous'

    suggested_action = 'one-hot encode (discrete)' if will_discrete else 'treat as continuous'
    sample_vals = list(pd.Series(col_vals.unique()).sort_values()[:6]) if num_unique > 0 else []

    rows.append({
        'Column': col,
        'Label': label_map.get(col, ''),
        'DataType': str(dtype),
        'NumUnique': num_unique,
        'IsString': dtype == 'object' or str(dtype).startswith('category'),
        'IsInteger': pd.api.types.is_integer_dtype(X[col].dtype),
        'WillBeDiscrete': will_discrete,
        'DecisionReason': reason,
        'SuggestedAction': suggested_action
    })

variable_summary = pd.DataFrame(rows)
# Order discrete first for visibility
variable_summary = variable_summary.sort_values(by=['WillBeDiscrete', 'NumUnique'], ascending=[False, True]).reset_index(drop=True)

print(f"Total variables: {len(variable_summary)}")
print(f"Discrete by rule: {int(variable_summary['WillBeDiscrete'].sum())}")
print(f"MAX_UNIQUE_DISCRETE = {MAX_UNIQUE_DISCRETE}")

pd.set_option('display.max_rows', None)
display(variable_summary)


Total variables: 302
Discrete by rule: 206
MAX_UNIQUE_DISCRETE = 30


,Column,Label,DataType,NumUnique,IsString,IsInteger,WillBeDiscrete,DecisionReason,SuggestedAction
0,intdatd_if,"Interview date: Day, imputation flag",int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
1,intdatm_if,"Interview date: Month, imputation flag",int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
2,intdaty_if,"Interview date: Year, imputation flag",int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
3,doby_if,DOB Year imputation flag,int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
4,age_if,Imputation flag for age_dv,int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
5,school_dv,Never went to/still at school,int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
6,sex,Sex,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
7,lkmove,Prefers to move house,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
8,xpmove,Expects to move in next year,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
9,health,Long-standing illness or disability,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)


## Scaling continuous variables

Great! Now we need to scale the continuous variables.

In [38]:
# Scale continuous variables to mean 0, std 1
from sklearn.preprocessing import StandardScaler
import numpy as np

# Identify continuous columns using the variable_summary decisions
continuous_cols = variable_summary.loc[~variable_summary['WillBeDiscrete'], 'Column'].tolist()

# Only keep columns that still exist in X (defensive)
continuous_cols = [c for c in continuous_cols if c in X.columns]

print(f'Found {len(continuous_cols)} continuous column(s) to scale')

# Keep a copy of the unscaled X in case we need it later
X_unscaled = X.copy()

scaler = None
if len(continuous_cols) > 0:
    scaler = StandardScaler()
    # Convert to float (safe) and scale in-place on a copy
    X_scaled = X.copy()
    try:
        X_scaled[continuous_cols] = scaler.fit_transform(X_scaled[continuous_cols].astype(float))
    except Exception as e:
        # Fall back to scaling each column separately if there are issues with mixed dtypes
        print('Warning: bulk scaling failed, falling back to per-column scaling. Error:', e)
        for col in continuous_cols:
            try:
                vals = X_scaled[col].astype(float).values.reshape(-1, 1)
                X_scaled[col] = scaler.fit_transform(vals).ravel()
            except Exception as e2:
                print(f'  Could not scale column {col}:', e2)
    
    # Replace X with scaled version
    X = X_scaled
else:
    print('No continuous columns to scale; X left unchanged')

print('Shape of X after scaling:', X.shape)

# Quick sanity checks
if scaler is not None and len(continuous_cols) > 0:
    # show means and stds (approx) for a few columns
    sample_check = continuous_cols[:6]
    means = X[sample_check].mean().round(6)
    stds = X[sample_check].std().round(6)
    print('Sample scaled means (should be near 0):')
    print(means.to_dict())
    print('Sample scaled stds (should be near 1):')
    print(stds.to_dict())

# Display head for quick verification
pd.set_option('display.max_columns', None)
display(X.head())

Found 96 continuous column(s) to scale
Shape of X after scaling: (106771, 302)
Sample scaled means (should be near 0):
{'prob01ni': 0.0, 'b_mortus_tw': -0.0, 'prob91e': -0.0, 'prob91w': -0.0, 'prob91s': -0.0, 'prob99w': 0.0}
Sample scaled stds (should be near 1):
{'prob01ni': 1.000005, 'b_mortus_tw': 1.000005, 'prob91e': 1.000005, 'prob91w': 1.000005, 'prob91s': 1.000005, 'prob99w': 1.000005}


,nbrsnci_dv,sex,dvage,istrtdatd,istrtdatm,istrtdaty,lkmove,xpmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,b_hidp,b_pno,b_ivfio,b_ivfho,b_month,c_hidp,c_pno,c_ivfio,c_ivfho,c_month,d_hidp,d_pno,d_ivfio,d_ivfho,d_month,e_hidp,e_ivfio,e_ivfho,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,genetics,epigenetics,xwdat_dv,scend_dv,school_dv,bornuk_dv,generation,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
8,-0.440636,2,-0.730395,1.841032,1,2018,2,2,2,1,1,2,2,0,0,0,0,0,0,1,2,1,2,3,1,1,1,1,1,0,1,1,1,5,5,3,6,2,1,2,0,0,1,0,0,0,0,0,2,0,2,0,4,1,19,-0.660555,0.433265,0,0,0.753119,0,1.093031,0,1.173913,0,-0.140384,0,-0.578007,-0.105704,-0.114112,2,-0.730616,1.841032,1,2018,0.910605,2,1,1,1,1,0,0,0,1,-0.083346,-0.094846,-0.096086,-0.305152,-0.510221,0.269495,1,8,2,1,0,8,4,7,1,0,0,2,20,1,2,1,1,2,1,1,4,4,0,0,2,2,0,0,1,4,0.187421,0.552690,0.290543,1,0,0,0,0,-0.316392,-0.279575,-0.412347,-0.112562,10,1.830483,1,2018,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2,2,1,1,0,1,0,1,1,2,1,1,1,1,1,1,1,1,1,1,0,1,0.723646,0.055377,-0.619351,3,0,1,0.578201,0,0.544842,0,1,0.663123,0.625091,-0.130702,-0.129852,-0.140547,0.991024,0.266870,0.767635,1.457195,0.485031,5,1,0,1,0,0,1,0,1,2,1,2,1,3,0.821956,2,0,1.340219,1,-0.527096,110,0.798744,0,2,14,2,14,2,0.120716,2,1,10,11,0.075896,2,2,11,11,0.074458,2,2,11,11,0.072513,1,10,0.071291,1,1,10,12,0,0,3,15,1,1,6,1,1,1,1,0.701625,-0.248481,-0.296052,-0.250578,-0.294834,-0.258463,-0.196618,-1.363109,-0.186445,-0.239297,0.085939,0.085984,0.086015,0.086112,0.086206,0.086267,0.086321,0.086364,0.1064

## Encoding categorical variables

Great, now we need to encode categorical variables using one-hot encoding

In [39]:
# Encode categorical variables using one-hot encoding, then show shape and head

# Identify categorical (discrete) columns from variable_summary
cat_cols = variable_summary.loc[variable_summary['WillBeDiscrete'], 'Column'].tolist()

# Defensive: keep only those present in X
cat_cols = [c for c in cat_cols if c in X.columns]
print(f'Found {len(cat_cols)} categorical column(s) to encode')

# If there are categorical columns, create dummies and merge with the rest of X
if len(cat_cols) > 0:
    # Convert to string to ensure stable dummy names (preserve distinct categories)
    cat_df = X[cat_cols].astype(str).apply(lambda s: s.str.replace(' ', '_'))
    # Create dummies; drop_first avoids creating a full-rank encoding
    dummies = pd.get_dummies(cat_df, prefix=cat_cols, prefix_sep='_', drop_first=True, dummy_na=False)
    # Build new X: drop original categorical columns, concat dummies
    X_encoded = X.drop(columns=cat_cols).copy()
    # Ensure no column name collisions (rename if necessary)
    overlap = set(X_encoded.columns).intersection(dummies.columns)
    if overlap:
        # Rare: if a dummy name collides with existing column, prefix dummy names with 'dum_'
        dummies = dummies.rename(columns={c: f'dum_{c}' for c in dummies.columns})
    X_encoded = pd.concat([X_encoded, dummies], axis=1)
else:
    print('No categorical columns selected for encoding')
    X_encoded = X.copy()

# Replace X with encoded version for downstream modeling
X = X_encoded

print('Shape of X after encoding:', X.shape)
pd.set_option('display.max_columns', None)
display(X.head())

Found 206 categorical column(s) to encode
Shape of X after encoding: (106771, 1457)


,nbrsnci_dv,dvage,istrtdatd,istrtdatmm,istrtdatss,fimngrs_dv,fimnlabgrs_dv,fimnlabnet_dv,fiyrinvinc_dv,fibenothr_dv,j2pay_dv,j2paynet_dv,age_dv,intdatd_dv,doby_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,intdated,xpfood1_g3,xpfdout_g3,xpaltob_g3,fihhmngrs_dv,fihhmnlabgrs_dv,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ieqmoecd_dv,fihhmngrs_if,aintlen,ivtnc,b_hidp,c_hidp,d_hidp,e_hidp,g_hidp,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd,sex_2,lkmove_2,xpmove_2,health_2,aidxhh_2,j2has_2,bensta2_1,bensta3_1,bensta4_1,bensta5_1,bensta6_1,bensta7_1,bensta96_1,vote1_2,mobuse_2,employ_2,jbhas_2,j2pay_if_1,fimngrs_tc_1,fimnlabgrs_tc_1,fimnlabnet_tc_1,fiyrinvinc_tc_1,fibenothr_tc_1,pensioner_dv_2,urban_dv_2,xtra5min_dv_1,livesp_dv_1,cohab_dv_1,single_dv_1,depchl_dv_2,respm16_dv_2,respf16_dv_2,rach16_dv_2,qfhighfl_dv_1,scflag_dv_1,paygu_if_1,paynu_if_1,seearngrs_if_1,fiyrinvinc_if_1,fihhmngrs_tc_1,fihhmnlabgrs_tc_1,fihhnegsei_if_1,w6osmflag_1,dcsedfl_dv_2,genetics_1,epigenetics_1,xwdat_dv_3,bornuk_dv_2,evercoh_dv_2,evermar_dv_2,anychild_dv_2,finfut_2,finfut_3,respf16_1,respf16_2,respm16_1,respm16_2,sex_dv_1,sex_dv_2,npn_dv_1,npn_dv_2,npns_dv_1,npns_dv_2,ngrp_dv_1,ngrp_dv_2,hhresp_dv_2,hhresp_dv_3,ivfho_11,ivfho_12,nonepar_dv_1,nonepar_dv_2,vote6_2,vote6_3,vote6_4,country_2,country_3,country_4,ivh10_-2,ivh10_-9,ivh10_0,ivh11_-2,ivh11_-9,ivh11_0,ivh12_-2,ivh12_-9,ivh12_0,ivh13_-2,ivh13_-9,ivh13_0,ivh14_-2,ivh14_-9,ivh14_0,ivh15_-2,ivh15_-9,ivh15_0,ivh16_-2,ivh16_-9,ivh16_0,ctband_if_1,ctband_if_2,ctband_if_3,outcome_210,outcome_211,outcome_214,ethn_dv_source_2,ethn_dv_source_3,ethn_dv_source_4,finnow_2,finnow_3,finnow_4,finnow_5,ivh1_-2,ivh1_-9,ivh1_0,ivh1_1,ivh2_-2,ivh2_-9,ivh2_0,ivh2_1,ivh3_-2,ivh3_-9,ivh3_0,ivh3_1,ivh4_-2,ivh4_-9,ivh4_0,ivh4_1,ivh5_-2,ivh5_-9,ivh5_0,ivh5_1,ivh6_-2,ivh6_-9,ivh6_0,ivh6_1,ivh7_-2,ivh7_-9,ivh7_0,ivh7_1,ivh8_-2,ivh8_-9,ivh8_0,ivh8_1,ivh9_-2,ivh9_-9,ivh9_0,ivh9_1,fuelhave1_-2,fuelhave1_-9,fuelhave1_0,fuelhave1_1,fuelhave2_-2,fuelhave2_-9,fuelhave2_0,fuelhave2_1,fuelhave3_-2,fuelhave3_-9,fuelhave3_0,fuelhave3_1,fuelhave4_-2,fuelhave4_-9,fuelhave4_0,fuelhave4_1,fuelhave96_-2,fuelhave96_-9,fuelhave96_0,fuelhave96_1,fuelduel_-2,fuelduel_-8,fuelduel_1,fuelduel_2,heatch_-2,heatch_-9,heatch_1,heatch_2,pcnet_-2,pcnet_-8,pcnet_1,pcnet_2,ncouple_dv_1,ncouple_dv_2,ncouple_dv_3,ncouple_dv_4,nch34_dv_0,nch34_dv_1,nch34_dv_2,nch34_dv_3,npens_dv_1,npens_dv_2,npens_dv_3,npens_dv_4,nch10to15_1,nch10to15_2,nch10to15_3,nch10to15_4,nch10to15_5,npensioner_dv_0,npensioner_dv_1,npensioner_dv_2,npensioner_dv_3,npensioner_dv_4,marstat_dv_2,marstat_dv_3,marstat_dv_4,marstat_dv_5,marstat_dv_6,hiqual_dv_2,hiqual_dv_3,hiqual_dv_4,hiqual_dv_5,hiqual_dv_9,xphsdct_-2,xphsdct_-8,xphsdct_-9,xphsdct_1,xphsdct_2,xphsdba_-2,xphsdba_-9,xphsdba_1,xphsdba_2,xphsdba_3,cduse1_-2,cduse1_-8,cduse1_-9,cduse1_0,cduse1_1,cduse2_-2,cduse2_-8,cduse2_-9,cduse2_0,cduse2_1,cduse5_-2,cduse5_-8,cduse5_-9,cduse5_0,cduse5_1,cduse6_-2,cduse6_-8,cduse6_-9,cduse6_0,cduse6_1,cduse7_-2,cduse7_-8,cduse7_-9,cduse7_0,cduse7_1,cduse8_-2,cduse8_-8,cduse8_-9,cduse8_0,cduse8_1,cduse9_-2,cduse9_-8,cduse9_-9,cduse9_0,cduse9_1,cduse12_-2,cduse12_-8,cduse12_-9,cduse12_0,cduse12_1,cduse13_-2,cduse13_-8,cduse13_-9,cduse13_0,cduse13_1,cduse96_-2,cduse96_-8,cduse96_-9,cduse96_0,cduse96_1,n10to15_1,n10to15_2,n10to15_3,n10to15_4,n10to15_5,nc

## Fitting the lasso model

Lets move on to fitting the lasso model!

In [40]:
# Build and run Lasso, then map nonzero coefficients back to variable labels (removed BaselineCategory)
from sklearn.linear_model import LassoCV
import numpy as np
import pandas as pd

# Fit Lasso with cross-validation
lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
lasso.fit(X, Y)

# Get nonzero coefficients sorted by absolute value
coefs = pd.Series(lasso.coef_, index=X.columns)
nonzero_coefs = coefs[coefs != 0].reindex(coefs[coefs != 0].abs().sort_values(ascending=False).index)

# Create label mapping from Stata metadata
label_map = dict(zip(meta.column_names, meta.column_labels)) if hasattr(meta, 'column_labels') else {}
original_vars = set(meta.column_names) if hasattr(meta, 'column_names') else set()

def parse_variable(varname):
    """Parse variable to get base variable, category, and labels (no baseline category)."""
    # Find base variable and category
    if varname in original_vars:
        base_var, category = varname, ''
    elif '_' in varname:
        base, cat = varname.rsplit('_', 1)
        if base in original_vars:
            base_var, category = base, cat
        else:
            # Try progressively shorter prefixes for variables with underscores
            parts = varname.split('_')
            base_var, category = varname, ''
            for i in range(len(parts) - 1, 0, -1):
                potential_base = '_'.join(parts[:i])
                if potential_base in original_vars:
                    base_var, category = potential_base, '_'.join(parts[i:])
                    break
    else:
        base_var, category = varname, ''

    # Get variable label
    variable_label = label_map.get(base_var, base_var)

    # Get category label from value labels
    category_label = ''
    if category and hasattr(meta, 'variable_value_labels') and base_var in meta.variable_value_labels:
        value_dict = meta.variable_value_labels[base_var]
        for cat_type in [int, float, str]:
            try:
                cat_key = cat_type(category)
                if cat_key in value_dict:
                    category_label = value_dict[cat_key]
                    break
            except (ValueError, TypeError):
                continue

    return base_var, variable_label, category, category_label

# Build results table (no BaselineCategory)
results_data = []
for var in nonzero_coefs.index:
    base_var, variable_label, category, category_label = parse_variable(var)
    results_data.append({
        'Variable': var,
        'Label': variable_label,
        'Category': category,
        'CategoryLabel': category_label,
        'Coefficient': nonzero_coefs[var]
    })

# Display results with formatted CategoryLabel and wider Coefficient display
result_df = pd.DataFrame(results_data)
pd.set_option('display.max_rows', None)
print(f"Lasso found {len(result_df)} significant variables (nonzero coefficients):")

# Prepare display copy: truncate long category labels and format coefficients for readability
max_cat_len = 40  # max characters to show for category labels
result_df['CategoryLabel'] = result_df['CategoryLabel'].astype(str).apply(
    lambda s: s if len(s) <= max_cat_len else s[:max_cat_len-3] + '...'
)

# Keep numeric coefficient for downstream use, but create a formatted display version
result_df['Coefficient'] = result_df['Coefficient'].astype(float)
result_df_display = result_df.copy()
result_df_display['Coefficient'] = result_df_display['Coefficient'].map(lambda v: f"{v: .6f}")

# Show a compact, clearly formatted table
display_cols = ['Variable', 'Label', 'Category', 'CategoryLabel', 'Coefficient']
pd.set_option('display.max_colwidth', 50)
display(result_df_display[display_cols])

Lasso found 206 significant variables (nonzero coefficients):


,Variable,Label,Category,CategoryLabel,Coefficient
0,sf12mcs_dv,SF-12 Mental Component Summary (MCS),,,-3.450475
1,sclfsato_7,Satisfaction with life overall,7,completely satisfied,-2.429473
2,finnow_5,Subjective financial situation - current,5,Finding it very difficult,2.008797
3,sclfsato_6,Satisfaction with life overall,6,mostly satisfied,-1.895336
4,sclfsato_5,Satisfaction with life overall,5,somewhat satisfied,-1.239503
5,finnow_4,Subjective financial situation - current,4,Finding it quite difficult,1.033134
6,sf12pcs_dv,SF-12 Physical Component Summary (PCS),,,-0.964695
7,sclfsato_4,Satisfaction with life overall,4,Neither Sat nor Dissat,-0.691199
8,finfut_2,Subjective financial situation - future,2,Worse of than now,0.572704
9,istrtdaty_2011,Individual interview start date (year),2011,,-0.381945


## Analysis

**Very interesting outcome! Here are my key takeaways:**

Already included in my refined variable list:

- Neighbourhood social cohesion (nbrsnci_dv) is confirmed to be correlated with subjective well-being (scghq1_dv), as expected
- Demographics are important (e.g. age, gender, ethnicity)
- Education is important (highest qualification)
- Current economic activity is important

Variables to consider adding:
- Subjective financial situation (finnow) (finfut) -- clearly subjective financial strain is a key predictor of well-being
- hhsize (household size)
- tenure_dv (housing tenure)

Variables I could consider deleting from my refined variable list:
- Job industry (not present here, and will be annoying due to lots of dummies)

Noteworthy variables:
- sclfsat* variables (e.g. satisfaction with life, health, income) are strong predictors of wellbeing. But they are effecitvely alternative wellbeing outcomes. If we include them, we risk circularity (i.e. we are predicting wellbeing with other measures of wellbeing). So I will exclude them
- sf12mcs (SF-12 mental health score) and sf12pcs (SF-12 physical health score) are strong predictors of wellbeing. They are also arguably alternative measures of wellbeing, so if we include them we risk circularity (over-control).

**Final list of variables to include in my refined variable list:**

- pidp : Cross-wave person identifier (public release)
- wave : 
- wave_num : 
- hidp : Household identifier (public release)

- nbrsnci_dv : Buckner's Neighbourhood Cohesion Instrument (**X**)
- gor_dv : Government Office Region
- urban_dv : Urban or rural area, derived
- sex_dv : Sex, derived
- age_dv : Age, derived from dob_dv and intdat_dv
- ethn_dv : Ethnic group (derived from multiple sources)
- marstat_dv : Harmonised de facto marital status
- jbstat : Current economic activity
- qfhigh_dv : Highest educational qualification ever reported
- fimnnet_dv : total net personal income - no deductions
- fihhmnnet1_dv : total household net income - no deductions
- houscost1_dv : monthly housing cost including mortgage principal payments
- health : Long-standing illness or disability
- scghq1_dv : Subjective wellbeing (GHQ): Likert (**Y**)


